In [ ]:
import os

# Import installed libraries
import numpy as np
import pandas as pd
import sklearn

from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [ ]:
try:
    from google.colab import drive
    drive.mount('/gdrive')
    core_path = "/path name/"
except ModuleNotFoundError:
    core_path = ""

Mounted at /gdrive


In [ ]:
asteroids_df = pd.read_pickle(os.path.join(core_path, "data/lvl2/", "asteroids.pkl"))

In [ ]:
asteroids_df.loc[:, "Class"] = asteroids_df["Main_Group"].apply(lambda x: 1 if x=="X" else 0)


In [ ]:
asteroids_X = np.array([k["Reflectance_norm550nm"].tolist() for k in asteroids_df["SpectrumDF"]])
asteroids_y = np.array(asteroids_df["Class"].to_list())

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

# Create a simple, single train / test split
for train_index, test_index in sss.split(asteroids_X, asteroids_y):

    X_train, X_test = asteroids_X[train_index], asteroids_X[test_index]
    y_train, y_test = asteroids_y[train_index], asteroids_y[test_index]

# Compute class weightning
positive_class_weight = int(1.0 / (sum(y_train) / len(X_train)))

In [ ]:
param_grid = [
  {'C': [1, 10, 100], 'kernel': ['linear']},
  {'C': [1, 10, 100], 'kernel': ['rbf']},
 ]

# Set the SVM classifier
svc = svm.SVC(class_weight={1: positive_class_weight})

# Instantiate the StandardScaler (mean 0, standard deviation 1) and use the training data to fit
# the scaler
scaler = preprocessing.StandardScaler().fit(X_train)

# Transform now the training data
X_train_scaled = scaler.transform(X_train)

# Set the GridSearch and ...
wclf = GridSearchCV(svc, param_grid, scoring='f1', verbose=3, cv=5)

# ... perform the training!
wclf.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ................C=1, kernel=linear;, score=0.553 total time=   0.2s
[CV 2/5] END ................C=1, kernel=linear;, score=0.555 total time=   0.2s
[CV 3/5] END ................C=1, kernel=linear;, score=0.585 total time=   0.1s
[CV 4/5] END ................C=1, kernel=linear;, score=0.512 total time=   0.2s
[CV 5/5] END ................C=1, kernel=linear;, score=0.577 total time=   0.3s
[CV 1/5] END ...............C=10, kernel=linear;, score=0.557 total time=   0.9s
[CV 2/5] END ...............C=10, kernel=linear;, score=0.559 total time=   0.7s
[CV 3/5] END ...............C=10, kernel=linear;, score=0.600 total time=   0.6s
[CV 4/5] END ...............C=10, kernel=linear;, score=0.571 total time=   0.3s
[CV 5/5] END ...............C=10, kernel=linear;, score=0.606 total time=   0.2s
[CV 1/5] END ..............C=100, kernel=linear;, score=0.562 total time=   2.1s
[CV 2/5] END ..............C=100, kernel=linear;,

GridSearchCV(cv=5, estimator=SVC(class_weight={1: 5}),
             param_grid=[{'C': [1, 10, 100], 'kernel': ['linear']},
                         {'C': [1, 10, 100], 'kernel': ['rbf']}],
             scoring='f1', verbose=3)

In [ ]:
final_clf = wclf.best_estimator_

In [ ]:
final_clf

SVC(C=100, class_weight={1: 5})

In [ ]:
X_test_scaled = scaler.transform(X_test)

# ... and perform a predicition
y_test_pred = final_clf.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_test_pred)

print(conf_mat)

# The order of the confusion matrix is:
#     - true negative (top left, tn)
#     - false positive (top right, fp)
#     - false negative (bottom left, fn)
#     - true positive (bottom right, tp)
tn, fp, fn, tp = conf_mat.ravel()

[[213   6]
 [  3  45]]


In [ ]:
recall_score = round(sklearn.metrics.recall_score(y_test, y_test_pred), 3)
print(f"Recall Score: {recall_score}")

# Precision: ratio of correctly classified X Class spectra, considering the false positives
# (precision = tp / (tp + fp))
precision_score = round(sklearn.metrics.precision_score(y_test, y_test_pred), 3)
print(f"Precision Score: {precision_score}")

# A combined score
f1_score = round(sklearn.metrics.f1_score(y_test, y_test_pred), 3)
print(f"F1 Score: {f1_score}")

Recall Score: 0.938
Precision Score: 0.882
F1 Score: 0.909
